### RatA14 rollipram SD recording info

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import analyses
from neuropy import plotting
import subjects

sess = subjects.sdrol.ratA14_2020_02_26[0]
print(sess.recinfo)


filename: /data/Clustering/sessions/rolipram/A14_2020-02-26_SDROL/A14_2020-02-26_SDROL.xml 
# channels: 256
sampling rate: 30000
lfp Srate (downsampled): 1250



In [8]:
sess.probegroup.get_disconnected_channels('shank')

AttributeError: 'ProbeGroup' object has no attribute 'get_disconnected_channels'

In [5]:
file = "/data/Clustering/sessions/rolipram/A14_2020-02-26_SDROL/A14_2020-02-26_SDROL.probe.npy"

data = np.load(file,allow_pickle=True).item()
data

{'data':           x       y  contact_id channel_id  shank_id connected  probe_id
 0       0.0     0.0         0.0         14       0.0      True       0.0
 1       0.0    20.0         1.0         12       0.0      True       0.0
 2       0.0    40.0         2.0         10       0.0      True       0.0
 3       0.0    60.0         3.0          8       0.0      True       0.0
 4       0.0    80.0         4.0          6       0.0      True       0.0
 ..      ...     ...         ...        ...       ...       ...       ...
 123  1565.0  2460.0       123.0        132       8.0     False       1.0
 124  1565.0  2480.0       124.0        131       8.0     False       1.0
 125  1565.0  2500.0       125.0        130       8.0     False       1.0
 126  1565.0  2520.0       126.0        129       8.0     False       1.0
 127  1565.0  2540.0       127.0        128       8.0     False       1.0
 
 [256 rows x 7 columns],
 'metadata': None}

### Set probe configuration
Since this is ratN. This animal had 8 shank probe from diagnostic biochips

In [ ]:
from neuropy.core import Shank, Probe, ProbeGroup

shanks = []
channel_groups = sess.recinfo.channel_groups
for i in range(8):

    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=20,
        y_shift_per_column=[0, -7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shanks.append(shank)
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)

probe1 = Probe(shanks)

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)

prbgrp.save(sess.filePrefix.with_suffix(".probegroup.npy"))
plotting.plot_probe(prbgrp)


### Experimental paradigm epochs
The epoch timings were provided by Jahangir (BGN_2019-10-19_SDROL.epoch.npy), so directly copying from that.

In [ ]:
import pandas as pd
from neuropy.core import Epoch

epochs = pd.DataFrame(
    {
        "start": [0, 10924, 14756, 14756, 32923],
        "stop": [10923, 14755, 53581, 32922, 51081],
        "label": ["pre", "maze", "post", "sd", "rs"],
    }
)

Epoch(epochs=epochs).save(sess.filePrefix.with_suffix(".paradigm.npy"))


### Artifacts
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [ ]:
signal = sess.eegfile.get_signal([20])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=10,edge_cutoff=2)
artifact_epochs.save(sess.filePrefix.with_suffix(".artifact.npy"))

In [ ]:
# sess.recinfo.write_epochs(sess.artifact)
signal = sess.eegfile.get_signal([20])
plotting.plot_artifact_epochs(sess.artifact, signal)

### Neurons
- Jahangir had already extracted clusters into .neurons.npy. However, some of the units have neuron_type labelled as '1'. I checked their autocorrelograms and based on their shape and frate all seemed like pyramidal neurons. I am re-labelling these units as 'pyr'. There were 30 such units.

In [ ]:
#autocorrelogram generation for unit labelled as '1'

from neuropy.utils import neurons_util

neurons = sess.neurons.get_neuron_type(['1'])
frate = neurons.firing_rate
acg = neurons_util.calculate_neurons_acg(neurons=neurons,window_size=0.1)

_,axs = plt.subplots(5,6)
axs = axs.reshape(-1)
for i,ax in enumerate(axs):
    ax.plot(acg[i,:])
    ax.set_title(frate[i].round(2))

In [ ]:
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

# file = "/data/Clustering/sessions/rolipram/BGN_2019-10-19_SDROL/BGN_2019-10-19_SDROL.neurons.npy"

og_neuron_type = sess.neurons.neuron_type

new_neuron_type = np.where(og_neuron_type==['1'],'pyr',og_neuron_type)


neurons = Neurons(
    np.array(sess.neurons.spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=sess.neurons.sampling_rate,
    peak_channels=sess.neurons.peak_channels,
    waveforms=sess.neurons.waveforms,
    shank_ids=sess.neurons.shank_ids,
    neuron_type=og_neuron_type,
)
neurons.save(sess.filePrefix.with_suffix(".neurons"))


/data/Clustering/sessions/rolipram/BGN_2019-10-19_SDROL/BGN_2019-10-19_SDROL.neurons saved
